In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
%cd /content/drive/MyDrive/IR_Project/News/

/content/drive/MyDrive/IR_Project/News


In [ ]:
import json
import pandas as pd
from pprint import pprint
from tqdm import tqdm

## **Data**

In [ ]:
with open('train_questions.json') as f:
    d = json.load(f)
data = pd.DataFrame.from_dict(d)

with open('train_outputs.json') as f:
    t = json.load(f)

outputs=pd.DataFrame.from_dict(t['golds'])
final_data=pd.merge(data, outputs, on='id')

In [ ]:
final_data.head(1)

,id,input,profile,output
0,300,Generate a headline for the following article:...,[{'text': 'Homeowners sell their homes and buy...,The Pre-Crisis HELOC Chickens Are Coming Home ...


In [ ]:
print(final_data.iloc[0])

id                                                       300
input      Generate a headline for the following article:...
profile    [{'text': 'Homeowners sell their homes and buy...
output     The Pre-Crisis HELOC Chickens Are Coming Home ...
Name: 0, dtype: object


In [ ]:
#making my testing sample
test_data=final_data.sample(100)

In [ ]:
test_data=pd.read_pickle('preprompt_100.pkl')

In [ ]:
test_data.columns

Index(['id', 'input', 'profile', 'output', 'bm25_results', 'keyword_results',
       'summary_results', 'style_results', 'input_keywords', 'input_summary',
       'prompts'],
      dtype='object')

In [ ]:
test_data=test_data[['id', 'input', 'profile', 'output', 'input_keywords', 'input_summary',]]

In [ ]:
len(test_data.iloc[3]['profile'])

1229

# STEP1: Retriever

### bm25

In [ ]:
!pip install langchain
!pip install rank_bm25

In [ ]:
from langchain.schema import Document
from langchain.retrievers import BM25Retriever

In [ ]:
def get_relevant_docs(query : str, docs: list):
  documents =  []

  for item in tqdm(docs):
      page = Document(page_content=item['text'],metadata = {"id" : item['id'], "title" : item['title']})
      documents.append(page)

  retriever = BM25Retriever.from_documents(documents)
  top_docs=retriever.get_relevant_documents(query)[:3]
  top3=[]
  for i in top_docs[:3]:
    res_dict={'text':i.page_content,'title':i.metadata['title'],'id':i.metadata['id']}
    top3.append(res_dict)
  return top3

In [ ]:
retriever_results=[]
for row in tqdm(range(test_data.shape[0])):
  input_text = test_data.iloc[row]['input']
  top_ret=get_relevant_docs(input_text, test_data.iloc[row]['profile'])
  retriever_results.append(top_ret)

100%|██████████| 100/100 [00:03<00:00, 27.20it/s]


In [ ]:
test_data['bm25_results']=retriever_results

### contriever

In [ ]:
!pip install --no-cache-dir transformers sentencepiece
!pip install faiss-gpu

In [ ]:
import numpy as np

In [ ]:
#got contriever results from python file on local machine

In [ ]:
test_data=pd.read_pickle('cont_res_100.pkl')

In [ ]:
test_data.iloc[0]['contriever']

[{'text': "Parents: We are not there to make life easy on our kids. It's our job to help them grow up with the skills to become successful adults.",
  'title': "Growing a Generation Who Doesn't know How to Fail",
  'id': '3019091003'},
 {'text': 'Teaching our kids to say "thank you" is important, but truly instilling a sense of gratitude in them is another matter entirely. Gratitude goes beyond good manners -- it\'s a mindset and a lifestyle.',
  'title': '11 Tips for Instilling True Gratitude in Your Kids',
  'id': '301909793'},
 {'text': "A friendly reminder from science that the flu vaccine isn't linked to autism.",
  'title': 'PSA: Pregnant Women Should Still Get Flu Shots',
  'id': '301909365'}]

# STEP 2 KEYWORD EXTRACTION

In [ ]:
!pip install keybert

  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 4.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for keybert: filename=keybert-0.8.3-py3-none-any.whl size=39124 sha256=b8617b7b1fe28938ec9b02c1a2e1ee86defe69ce8ac1b6d2c83f083563a388e8
  Stored in directory: /root/.cache/pip/wheels/70/88/07/1a3bc11fd1dd5f89924a02dcbca89a3015e25e8faa31f904dc
  Created wheel for sentence-transformers: filename=sentence_transformers-2.2.2-py3-none-any.whl size=125923 sha256=9d5f948c9d47203b7cbc4f77938bc4b7dda5cf7b7c3cbb1680b11cb735a6c89e
  Stored in directory: /root/.cache/pip/wheels/62/f2/10/1e606fd5f02395388f74e7462910fe851042f97238cbbd902f
Successfully built keybert sentence-transformers


In [ ]:
from keybert import KeyBERT
kw_model = KeyBERT()

.gitattributes:   0%|          | 0.00/1.18k [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.6k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/612 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

...

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

train_script.py:   0%|          | 0.00/13.2k [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

In [1]:
#kw_model.extract_keywords(doc, stop_words='english',use_mmr=True, diversity=0.3)

In [ ]:
keyword_results=[]
for row in tqdm(range(test_data.shape[0])):
  #cur_list = test_data.iloc[row]['bm25_results'] #ifbm25 run this
  cur_list = test_data.iloc[row]['contriever']
  updated_list = [{'text': entry['text'], 'title': entry['title'], 'id': entry['id'],'keywords': kw_model.extract_keywords(entry['text'],, stop_words='english',use_mmr=True, diversity=0.3)} for entry in cur_list]

  #updated_list = [{'text': entry['text'], 'title': entry['title'], 'id': entry['id'],'keywords': kw_model.extract_keywords(entry['text'])} for entry in cur_list]
  keyword_results.append(updated_list)

100%|██████████| 100/100 [00:22<00:00,  4.51it/s]


In [ ]:
test_data['keyword_results']=keyword_results

In [ ]:
test_data.head()

,id,input,profile,output,input_keywords,input_summary,contriever,keyword_results
1909,301909,Generate a headline for the following article:...,"[{'text': 'Bryant's widow testified that she ""...",33 Phrases To Stop Your Toddler's Tantrums,"tantrums, toddler, parenting, cooperation, smile",Here are some parenting phrases that can help ...,[{'text': 'Parents: We are not there to make l...,[{'text': 'Parents: We are not there to make l...
7215,307215,Generate a headline for the following article:...,[{'text': '5. Coming with your significant oth...,Beyonce: Met Gala 2013 Honorary Chair!,"beyonce, gala, chair, costume, institute",Do you think Beyonce is a good choice to co-ch...,[{'text': 'Editor's Note: Previous references ...,[{'text': 'Editor's Note: Previous references ...
693,30693,Generate a headline for the following article:...,"[{'text': '""Drivin down the wrong side of the ...",FEARLESS MEMOIR: 'World's Fair' (Chapter 18),"feet, claws, fear, terror, budge",My feet refused to move; they were glued to th...,[{'text': 'All I could think about was Daddy. ...,[{'text': 'All I could think about was Daddy. ...
2837,302837,Generate a headline for the following article:...,[{'text': 'He was subdued by passengers and cr...,Buzz Aldrin Looked As Baffled As We Felt Durin...,"infinity, right, don, know, really","""This is infinity here. It could be infinity. ...","[{'text': '""The article is an absolutely disgu...","[{'text': '""The article is an absolutely disgu..."
10676,3010676,Generate a headline for the following article:...,[{'text': 'Certain points during your day can ...,Starbucks To Stop Throwing Out Perfectly Good ...,"donations, coffee, employees, food, asking",The coffee chain says its employees called for...,[{'text': 'This is a brutal reminder that glob...,[{'text': 'This is a brutal reminder that glob...


In [ ]:
test_data.to_pickle('contriever/test_data_cont_100.pkl')

#  STEP 3 SUMMARIZER

In [ ]:
test_data=pd.read_pickle('contriever/test_data_cont_100.pkl')

In [ ]:
!pip install --no-cache-dir transformers sentencepiece

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 12.8 MB/s eta 0:00:00


In [ ]:
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer
import torch

# Ensure CUDA is available and set the device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Path to your downloaded model
model_path = "../pegasus-cnn_dailymail"
# Load the tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_path)
model = AutoModelForSeq2SeqLM.from_pretrained(model_path).to(device).half()

Some weights of PegasusForConditionalGeneration were not initialized from the model checkpoint at ../pegasus-cnn_dailymail and are newly initialized: ['model.decoder.embed_positions.weight', 'model.encoder.embed_positions.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
# TEST
raw_document = 'Homeowners sell their homes and buy other homes for a variety of reasons including a need to live closer to a place of employment, to be closer to family, to enjoy a better climate, or simply to upgrade. This article is about finding the best sequence of steps in the process.'

# Encode the text, ensuring that it is truncated to the maximum length acceptable by the model
input_ids = tokenizer.encode(raw_document, return_tensors="pt", truncation=True, max_length=model.config.max_position_embeddings).to(device)

# Perform inference
with torch.no_grad():
    output = model.generate(
        input_ids,
        num_beams=5,
        min_length=5,
        no_repeat_ngram_size=3,
        max_length=50
    )

# Decode and print the output
decoded_output = tokenizer.decode(output[0], skip_special_tokens=True)
print(decoded_output)

Homeowners sell their homes and buy other homes for a variety of reasons.<n>This article is about finding the best sequence of steps in the process.


In [ ]:
def get_summ(raw_document):
  # Encode the text, ensuring that it is truncated to the maximum length acceptable by the model
  input_ids = tokenizer.encode(raw_document, return_tensors="pt", truncation=True, max_length=model.config.max_position_embeddings).to(device)

  # Perform inference
  with torch.no_grad():
      output = model.generate(
          input_ids,
          num_beams=5,
          min_length=5,
          no_repeat_ngram_size=3,
          max_length=50
      )

  # Decode and print the output
  decoded_output = tokenizer.decode(output[0], skip_special_tokens=True)
  return decoded_output

In [ ]:
import pickle

In [ ]:
summ_results=[]
for row in tqdm(range(test_data.shape[0])):
  cur_list = test_data.iloc[row]['keyword_results']
  updated_list = [{'text': entry['text'], 'title': entry['title'], 'id': entry['id'],'keywords': entry['keywords'],'summary':get_summ(entry['text'])} for entry in cur_list]
  summ_results.append(updated_list)
  if(row%10==0):
    # Save to a pickle file
    with open("contriever/data_summ.pkl", "wb") as file:
      pickle.dump(summ_results, file)

100%|██████████| 100/100 [06:18<00:00,  3.79s/it]


In [ ]:
summ_results[0]

[{'text': "Parents: We are not there to make life easy on our kids. It's our job to help them grow up with the skills to become successful adults.",
  'title': "Growing a Generation Who Doesn't know How to Fail",
  'id': '3019091003',
  'keywords': [('parents', 0.5985),
   ('kids', 0.4438),
   ('adults', 0.4359),
   ('life', 0.345),
   ('skills', 0.3163)],
  'summary': "Parents: We are not there to make life easy on our kids.<n>It's our job to help them grow up with the skills to become successful adults."},
 {'text': 'Teaching our kids to say "thank you" is important, but truly instilling a sense of gratitude in them is another matter entirely. Gratitude goes beyond good manners -- it\'s a mindset and a lifestyle.',
  'title': '11 Tips for Instilling True Gratitude in Your Kids',
  'id': '301909793',
  'keywords': [('gratitude', 0.59),
   ('manners', 0.3924),
   ('kids', 0.2494),
   ('important', 0.1572),
   ('thank', 0.1485)],
  'summary': 'Teaching kids to say "thank you" is importa

In [ ]:
test_data['summary_results']=summ_results

In [ ]:
test_data.to_pickle('contriever/test_data_100_summ.pkl')

# STEP 4 STYLE ANALYSIS

##### Style Functions

In [ ]:
import collections as coll
import math
import pickle
import string
import numpy as np
from nltk.corpus import cmudict
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize, sent_tokenize
import nltk

nltk.download('cmudict')
nltk.download('stopwords')
nltk.download('punkt')

[nltk_data] Downloading package cmudict to /root/nltk_data...
[nltk_data]   Unzipping corpora/cmudict.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [ ]:
def syllable_count_Manual(word):
    word = word.lower()
    count = 0
    vowels = "aeiouy"
    if word[0] in vowels:
        count += 1
    for index in range(1, len(word)):
        if word[index] in vowels and word[index - 1] not in vowels:
            count += 1
            if word.endswith("e"):
                count -= 1
    if count == 0:
        count += 1
    return count


# ---------------------------------------------------------------------
# COUNTS NUMBER OF SYLLABLES

def syllable_count(word):
    global cmuDictionary
    d = cmuDictionary
    try:
        syl = [len(list(y for y in x if y[-1].isdigit())) for x in d[word.lower()]][0]
    except:
        syl = syllable_count_Manual(word)
    return syl

    # ----------------------------------------------------------------------------


# removing stop words plus punctuation.
def Avg_wordLength(str):
    str.translate(string.punctuation)
    tokens = word_tokenize(str, language='english')
    st = [",", ".", "'", "!", '"', "#", "$", "%", "&", "(", ")", "*", "+", "-", ".", "/", ":", ";", "<", "=", '>', "?",
          "@", "[", "\\", "]", "^", "_", '`', "{", "|", "}", '~', '\t', '\n']
    stop = stopwords.words('english') + st
    words = [word for word in tokens if word not in stop]
    return np.average([len(word) for word in words])


# ----------------------------------------------------------------------------


# returns avg number of characters in a sentence
def Avg_SentLenghtByCh(text):
    tokens = sent_tokenize(text)
    return np.average([len(token) for token in tokens])


# ----------------------------------------------------------------------------

# returns avg number of words in a sentence
def Avg_SentLenghtByWord(text):
    tokens = sent_tokenize(text)
    return np.average([len(token.split()) for token in tokens])


# ----------------------------------------------------------------------------


# GIVES NUMBER OF SYLLABLES PER WORD
def Avg_Syllable_per_Word(text):
    tokens = word_tokenize(text, language='english')
    st = [",", ".", "'", "!", '"', "#", "$", "%", "&", "(", ")", "*", "+", "-", ".", "/", ":", ";", "<", "=", '>', "?",
          "@", "[", "\\", "]", "^", "_", '`', "{", "|", "}", '~', '\t', '\n']
    stop = stopwords.words('english') + st
    words = [word for word in tokens if word not in stop]
    syllabls = [syllable_count(word) for word in words]
    p = (" ".join(words))
    return sum(syllabls) / max(1, len(words))


# -----------------------------------------------------------------------------

# COUNTS SPECIAL CHARACTERS NORMALIZED OVER LENGTH OF CHUNK
def CountSpecialCharacter(text):
    st = ["#", "$", "%", "&", "(", ")", "*", "+", "-", "/", "<", "=", '>',
          "@", "[", "\\", "]", "^", "_", '`', "{", "|", "}", '~', '\t', '\n']
    count = 0
    for i in text:
        if (i in st):
            count = count + 1
    return count / len(text)


# ----------------------------------------------------------------------------

def CountPuncuation(text):
    st = [",", ".", "'", "!", '"', ";", "?", ":", ";"]
    count = 0
    for i in text:
        if (i in st):
            count = count + 1
    return float(count) / float(len(text))


# ----------------------------------------------------------------------------
# RETURNS NORMALIZED COUNT OF FUNCTIONAL WORDS FROM A Framework for
# Authorship Identification of Online Messages: Writing-Style Features and Classification Techniques

def CountFunctionalWords(text):
    functional_words = """a between in nor some upon
    about both including nothing somebody us
    above but inside of someone used
    after by into off something via
    all can is on such we
    although cos it once than what
    am do its one that whatever
    among down latter onto the when
    an each less opposite their where
    and either like or them whether
    another enough little our these which
    any every lots outside they while
    anybody everybody many over this who
    anyone everyone me own those whoever
    anything everything more past though whom
    are few most per through whose
    around following much plenty till will
    as for must plus to with
    at from my regarding toward within
    be have near same towards without
    because he need several under worth
    before her neither she unless would
    behind him no should unlike yes
    below i nobody since until you
    beside if none so up your
    """

    functional_words = functional_words.split()
    words = RemoveSpecialCHs(text)
    count = 0

    for i in text:
        if i in functional_words:
            count += 1

    return count / len(words)


# ---------------------------------------------------------------------------

# also returns Honore Measure R
def hapaxLegemena(text):
    words = RemoveSpecialCHs(text)
    V1 = 0
    # dictionary comprehension . har word kay against value 0 kardi
    freqs = {key: 0 for key in words}
    for word in words:
        freqs[word] += 1
    for word in freqs:
        if freqs[word] == 1:
            V1 += 1
    N = len(words)
    V = float(len(set(words)))
    R = 100 * math.log(N) / max(1, (1 - (V1 / V)))
    h = V1 / N
    return R, h


# ---------------------------------------------------------------------------

def hapaxDisLegemena(text):
    words = RemoveSpecialCHs(text)
    count = 0
    # Collections as coll Counter takes an iterable collapse duplicate and counts as
    # a dictionary how many equivelant items has been entered
    freqs = coll.Counter()
    freqs.update(words)
    for word in freqs:
        if freqs[word] == 2:
            count += 1

    h = count / float(len(words))
    S = count / float(len(set(words)))
    return S, h


# ---------------------------------------------------------------------------

# c(w)  = ceil (log2 (f(w*)/f(w))) f(w*) frequency of most commonly used words f(w) frequency of word w
# measure of vocabulary richness and connected to zipfs law, f(w*) const rak kay zips law say rank nikal rahay hein
def AvgWordFrequencyClass(text):
    words = RemoveSpecialCHs(text)
    # dictionary comprehension . har word kay against value 0 kardi
    freqs = {key: 0 for key in words}
    for word in words:
        freqs[word] += 1
    maximum = float(max(list(freqs.values())))
    return np.average([math.floor(math.log((maximum + 1) / (freqs[word]) + 1, 2)) for word in words])


# --------------------------------------------------------------------------
# TYPE TOKEN RATIO NO OF DIFFERENT WORDS / NO OF WORDS
def typeTokenRatio(text):
    words = word_tokenize(text)
    return len(set(words)) / len(words)


# --------------------------------------------------------------------------
# logW = V-a/log(N)
# N = total words , V = vocabulary richness (unique words) ,  a=0.17
# we can convert into log because we are only comparing different texts
def BrunetsMeasureW(text):
    words = RemoveSpecialCHs(text)
    a = 0.17
    V = float(len(set(words)))
    N = len(words)
    B = (V - a) / (math.log(N))
    return B


# ------------------------------------------------------------------------
def RemoveSpecialCHs(text):
    text = word_tokenize(text)
    st = [",", ".", "'", "!", '"', "#", "$", "%", "&", "(", ")", "*", "+", "-", ".", "/", ":", ";", "<", "=", '>', "?",
          "@", "[", "\\", "]", "^", "_", '`', "{", "|", "}", '~', '\t', '\n']

    words = [word for word in text if word not in st]
    return words


# -------------------------------------------------------------------------
# K  10,000 * (M - N) / N**2
# , where M  Sigma i**2 * Vi.
def YulesCharacteristicK(text):
    words = RemoveSpecialCHs(text)
    N = len(words)
    freqs = coll.Counter()
    freqs.update(words)
    vi = coll.Counter()
    vi.update(freqs.values())
    M = sum([(value * value) * vi[value] for key, value in freqs.items()])
    K = 10000 * (M - N) / math.pow(N, 2)
    return K


# -------------------------------------------------------------------------


# -1*sigma(pi*lnpi)
# Shannon and sympsons index are basically diversity indices for any community
def ShannonEntropy(text):
    words = RemoveSpecialCHs(text)
    lenght = len(words)
    freqs = coll.Counter()
    freqs.update(words)
    arr = np.array(list(freqs.values()))
    distribution = 1. * arr
    distribution /= max(1, lenght)
    import scipy as sc
    H = sc.stats.entropy(distribution, base=2)
    # H = sum([(i/lenght)*math.log(i/lenght,math.e) for i in freqs.values()])
    return H


# ------------------------------------------------------------------
# 1 - (sigma(n(n - 1))/N(N-1)
# N is total number of words
# n is the number of each type of word
def SimpsonsIndex(text):
    words = RemoveSpecialCHs(text)
    freqs = coll.Counter()
    freqs.update(words)
    N = len(words)
    n = sum([1.0 * i * (i - 1) for i in freqs.values()])
    D = 1 - (n / (N * (N - 1)))
    return D


# ------------------------------------------------------------------

def FleschReadingEase(text, NoOfsentences):
    words = RemoveSpecialCHs(text)
    l = float(len(words))
    scount = 0
    for word in words:
        scount += syllable_count(word)

    I = 206.835 - 1.015 * (l / float(NoOfsentences)) - 84.6 * (scount / float(l))
    return I


# -------------------------------------------------------------------
def FleschCincadeGradeLevel(text, NoOfSentences):
    words = RemoveSpecialCHs(text)
    scount = 0
    for word in words:
        scount += syllable_count(word)

    l = len(words)
    F = 0.39 * (l / NoOfSentences) + 11.8 * (scount / float(l)) - 15.59
    return F


# -----------------------------------------------------------------
def dale_chall_readability_formula(text, NoOfSectences):
    words = RemoveSpecialCHs(text)
    difficult = 0
    adjusted = 0
    NoOfWords = len(words)
    with open('dale-chall.pkl', 'rb') as f:
        fimiliarWords = pickle.load(f)
    for word in words:
        if word not in fimiliarWords:
            difficult += 1
    percent = (difficult / NoOfWords) * 100
    if (percent > 5):
        adjusted = 3.6365
    D = 0.1579 * (percent) + 0.0496 * (NoOfWords / NoOfSectences) + adjusted
    return D


# ------------------------------------------------------------------
def GunningFoxIndex(text, NoOfSentences):
    words = RemoveSpecialCHs(text)
    NoOFWords = float(len(words))
    complexWords = 0
    for word in words:
        if (syllable_count(word) > 2):
            complexWords += 1

    G = 0.4 * ((NoOFWords / NoOfSentences) + 100 * (complexWords / NoOFWords))
    return G


# ------------------------------------------------------------------

#### Calls

In [ ]:
cmuDictionary = None
def FeatureExtraction(chunk, winSize):
    global cmuDictionary
    cmuDictionary = cmudict.dict()

    features = []

    # LEXICAL FEATURES
    mean_word_length = Avg_wordLength(chunk)
    features.append(f"Mean Word Length: {mean_word_length}")

    mean_sentence_length_chars = Avg_SentLenghtByCh(chunk)
    features.append(f"Mean Sentence Length (characters): {mean_sentence_length_chars}")

    mean_sentence_length_words = Avg_SentLenghtByWord(chunk)
    features.append(f"Mean Sentence Length (words): {mean_sentence_length_words}")

    mean_syllables_per_word = Avg_Syllable_per_Word(chunk)
    features.append(f"Mean Syllables per Word: {mean_syllables_per_word}")

    count_special_characters = CountSpecialCharacter(chunk)
    features.append(f"Count of Special Characters: {count_special_characters}")

    count_punctuation = CountPuncuation(chunk)
    features.append(f"Count of Punctuation: {count_punctuation}")

    count_functional_words = CountFunctionalWords(chunk)
    features.append(f"Count of Functional Words: {count_functional_words}")

    # VOCABULARY RICHNESS FEATURES
    type_token_ratio = typeTokenRatio(chunk)
    features.append(f"Type-Token Ratio: {type_token_ratio}")

    honore_measure_r, hapax_legemena = hapaxLegemena(chunk)
    features.append(f"Hapax Legemena: {hapax_legemena}, Honore Measure R: {honore_measure_r}")

    sicheles_measure_s, hapax_dis_legemena = hapaxDisLegemena(chunk)
    features.append(f"Hapax Dis Legemena: {hapax_dis_legemena}, Sicheles Measure S: {sicheles_measure_s}")

    yules_characteristic_k = YulesCharacteristicK(chunk)
    features.append(f"Yules Characteristic K: {yules_characteristic_k}")

    simpsons_index = SimpsonsIndex(chunk)
    features.append(f"Simpsons Index: {simpsons_index}")

    brunets_measure_w = BrunetsMeasureW(chunk)
    features.append(f"Brunets Measure W: {brunets_measure_w}")

    shannon_entropy = ShannonEntropy(chunk)
    features.append(f"Shannon Entropy: {shannon_entropy}")

    # READABILITY FEATURES
    flesch_reading_ease = FleschReadingEase(chunk, winSize)
    features.append(f"Flesch Reading Ease: {flesch_reading_ease}")

    flesch_cincade_grade_level = FleschCincadeGradeLevel(chunk, winSize)
    features.append(f"Flesch Cincade Grade Level: {flesch_cincade_grade_level}")

    #dale_chall_readability = dale_chall_readability_formula(chunk, winSize)
    #features.append(f"Dale Chall Readability Formula: {dale_chall_readability}")

    gunning_fox_index = GunningFoxIndex(chunk, winSize)
    features.append(f"Gunning Fox Index: {gunning_fox_index}")

    return '\n'.join(features)



def FeatureExtraction_titles(chunk_list):
    global cmuDictionary
    cmuDictionary = cmudict.dict()

    features = []

    # Initialize dictionaries to store sum and count for averaging
    sum_dict = {}
    count_dict = {}

    # Function to update sum and count dictionaries
    def update_dicts(feature_name, value):
        if feature_name not in sum_dict:
            sum_dict[feature_name] = value
            count_dict[feature_name] = 1
        else:
            sum_dict[feature_name] += value
            count_dict[feature_name] += 1

    # Iterate over each chunk in the list
    for chunk in chunk_list:
        # LEXICAL FEATURES
        update_dicts("Mean Word Length", Avg_wordLength(chunk))
        update_dicts("Mean Sentence Length (characters)", Avg_SentLenghtByCh(chunk))
        update_dicts("Mean Sentence Length (words)", Avg_SentLenghtByWord(chunk))
        update_dicts("Mean Syllables per Word", Avg_Syllable_per_Word(chunk))
        update_dicts("Count of Special Characters", CountSpecialCharacter(chunk))
        update_dicts("Count of Punctuation", CountPuncuation(chunk))
        update_dicts("Count of Functional Words", CountFunctionalWords(chunk))

        # VOCABULARY RICHNESS FEATURES
        update_dicts("Type-Token Ratio", typeTokenRatio(chunk))
        honore_measure_r, hapax_legemena = hapaxLegemena(chunk)
        update_dicts("Hapax Legemena", hapax_legemena)
        update_dicts("Honore Measure R", honore_measure_r)
        sicheles_measure_s, hapax_dis_legemena = hapaxDisLegemena(chunk)
        update_dicts("Hapax Dis Legemena", hapax_dis_legemena)
        update_dicts("Sicheles Measure S", sicheles_measure_s)
        update_dicts("Yules Characteristic K", YulesCharacteristicK(chunk))
        update_dicts("Simpsons Index", SimpsonsIndex(chunk))
        update_dicts("Brunets Measure W", BrunetsMeasureW(chunk))
        update_dicts("Shannon Entropy", ShannonEntropy(chunk))

        # READABILITY FEATURES
        update_dicts("Flesch Reading Ease", FleschReadingEase(chunk, len(chunk)))
        update_dicts("Flesch Cincade Grade Level", FleschCincadeGradeLevel(chunk,len(chunk)))
        #update_dicts("Dale Chall Readability Formula", dale_chall_readability_formula(chunk,len(chunk)))
        update_dicts("Gunning Fox Index", GunningFoxIndex(chunk, len(chunk)))

    # Calculate the average for each feature
    for feature_name, feature_sum in sum_dict.items():
        avg_value = feature_sum / count_dict[feature_name]
        features.append(f"{feature_name}: {avg_value}")

    return '\n'.join(features)

#### Test

In [ ]:
exp='''
Lexical Features:
1. Mean Word Length: Average number of characters in words.
2. Mean Sentence Length (characters): Average number of characters in sentences.
3. Mean Sentence Length (words): Average number of words in sentences.
4. Mean Syllables per Word: Average number of syllables in words.
5. Count of Special Characters: Normalized count of special characters over the length of the chunk.
6. Count of Punctuation: Normalized count of punctuation marks over the length of the chunk.
7. Count of Functional Words: Normalized count of functional words from a specific set.

Vocabulary Richness Features:
8. Type-Token Ratio: Ratio of different words to total words.
9. Hapax Legemena: Ratio of words that occur only once to total words.
10. Sicheles Measure S: Ratio of words that occur twice to total words.
11. Yules Characteristic K: Measure of vocabulary richness and connection to Zipf's Law.
12. Simpsons Index: Diversity index based on word frequency.
13. Brunets Measure W: Vocabulary richness index.
14. Shannon Entropy: Information entropy based on word frequency.

Readability Features:
15. Flesch Reading Ease: Readability score based on average sentence length and syllables per word.
16. Flesch Cincade Grade Level: Readability score based on average sentence length and syllables per word.
17. Dale Chall Readability Formula: Readability score considering difficult words.
18. Gunning Fox Index: Readability index based on sentence length and complex words.

'''

#### LLM BASED

In [ ]:
## PROMPT for LLM BASE WRITING STYLE

In [ ]:
!pip install google-generativeai

In [ ]:
import google.generativeai as palm
import os
os.environ['API_KEY']=#API KEY HERE
palm.configure(api_key=os.environ['API_KEY'])

## RUN STYLE ANALYSIS ON TEST DATA

In [ ]:
import time

In [ ]:
def get_palm_response(input_text, prompt_instructions):
  p="input text: "+ input_text+"\n"+prompt_instructions
  response = palm.generate_text(prompt=p)
  return response.result

In [ ]:
def get_writing_style(text,prompt_instructions,all_titles):
  statistical_analysis= FeatureExtraction_titles(all_titles)
  llm_bases_analysis=get_palm_response(text, prompt_instructions)

  return {'statistical_analysis':statistical_analysis,'llm_bases_analysis':llm_bases_analysis }

In [ ]:
prompt_instructions='''You are a writing style detector. Your job is to specify a writing style of input text above. When you do so, you describe outloud:
general writing style (like creative, non-fiction),
genres,
style descriptors (like full of dialogue, short sentences),
moods (like serious, joking),
authors that share a similar style (like Kurt Vonnegut),
common phrases or speech patterns. You are not to provide any other additional information.
'''
style_results=[]
for row in tqdm(range(test_data.shape[0])):
  cur_list = test_data.iloc[row]['summary_results']

  all_titles = [entry['title'] for entry in cur_list]
  all_profile_article = [entry['title']+'\n'+entry['text'] for entry in cur_list]

  style=get_writing_style("\n".join(all_profile_article),prompt_instructions,all_titles)
  style_results.append(style)

  if(row%10==0):
    # Save to a pickle file
    with open("style_checkpoints/cont_style_summ_"+str(row)+".pkl", "wb") as file:
      pickle.dump(style_results, file)
      time.sleep(30)

100%|██████████| 100/100 [09:12<00:00,  5.53s/it]


In [ ]:
test_data['style_results']=style_results
test_data.to_pickle('contriever/test_data_100_style.pkl')

In [ ]:
test_data.iloc[0]['style_results']

{'statistical_analysis': 'Mean Word Length: 4.928571428571428\nMean Sentence Length (characters): 48.333333333333336\nMean Sentence Length (words): 8.666666666666666\nMean Syllables per Word: 1.4404761904761905\nCount of Special Characters: 0.0\nCount of Punctuation: 0.014049097900029575\nCount of Functional Words: 0.5425925925925926\nType-Token Ratio: 1.0\nHapax Legemena: 1.0\nHonore Measure R: 219.30837373367004\nHapax Dis Legemena: 0.0\nSicheles Measure S: 0.0\nYules Characteristic K: 8879.62962962963\nSimpsons Index: 1.0\nBrunets Measure W: 4.017751666462215\nShannon Entropy: 3.163951032109891\nFlesch Reading Ease: 91.1828784679089\nFlesch Cincade Grade Level: 0.5873541227118864\nGunning Fox Index: 4.370695586447138',
 'llm_bases_analysis': 'general writing style: non-fiction\ngenres: opinion piece, parenting advice, health advice\nstyle descriptors: informal, conversational, personal, authoritative\nmoods: serious, informative\nauthors that share a similar style: Amy Poehler, Chel

## Adding keywords and summary for input too.

In [ ]:
keyword_list=[]
summary_list=[]
for row in tqdm(range(test_data.shape[0])):
  input = test_data.iloc[row]['input'].split("Generate a headline for the following article:")[-1].strip()
  keywords_scores=kw_model.extract_keywords(input)
  keyword_list.append(', '.join([kw[0] for kw in keywords_scores]))
  summary_list.append(get_summ(input))


In [ ]:
test_data['input_keywords']=keyword_list
test_data["input_summary"]=summary_list

In [ ]:
test_data.to_pickle('contriever/preprompt_100.pkl')

# COMPLETE PIPELINE

In [ ]:
preprompt_data=pd.read_pickle("contriever/preprompt_100.pkl")

In [ ]:
preprompt_data.head(1)

,id,input,profile,output,input_keywords,input_summary,contriever,keyword_results,summary_results,style_results
1909,301909,Generate a headline for the following article:...,"[{'text': 'Bryant's widow testified that she ""...",33 Phrases To Stop Your Toddler's Tantrums,"tantrums, toddler, parenting, cooperation, smile",Here are some parenting phrases that can help ...,[{'text': 'Parents: We are not there to make l...,[{'text': 'Parents: We are not there to make l...,[{'text': 'Parents: We are not there to make l...,{'statistical_analysis': 'Mean Word Length: 4....


In [ ]:
def generate_title_prompt(input_text, style_results, summary_results,ip_keys,ip_summ):
    # Extracting style analysis and general writing style of the user
    style_analysis = style_results['statistical_analysis']
    general_style = style_results['llm_bases_analysis']

    # Extracting information from previous articles
    previous_article_prompts = []
    for article in summary_results:
        article_text = article['text']
        title = article['title']
        keywords = ', '.join([kw[0] for kw in article['keywords']])
        summary = article['summary']
        previous_article_prompt = (
            f"Article Text: {article_text}\n"
            f"Title: {title}\n"
            f"Keywords: {keywords}\n"
            f"Summary: {summary}\n\n"
        )
        previous_article_prompts.append(previous_article_prompt)

    # Combining all information into the final prompt
    final_prompt_details = (
        f"Previous Articles Analysis:\n{''.join(previous_article_prompts)}"
        f"Statistical Writing Style Analysis of Titles:\n{style_analysis}\n\n"
        f"General Writing Style of User:\n{general_style}\n\n"
        f"New Input Article:\nArticle Text: {input_text}\n"
        f"Keywords: {ip_keys}\n"
        f"Summary: {ip_summ}\n\n"
    )

    final_prompt="Your task is to analyze the provided data to understand the user's writing style and approach to titling. This includes examining previous articles with their titles, keywords, and summaries, a statistical analysis of the user's writing style in their titles, a general characterization of their writing style, and the text of a new input article. Based on this analysis, generate a title for the new article that aligns with the user's unique style."\
    +"\n"\
    +final_prompt_details\
    +"\n"+"After reviewing all these elements, create a title for the new article that reflects the user's unique writing style and titling approach, as well as being pertinent to the new article's content and theme"

    return final_prompt


In [ ]:
# Example usage
input_article = preprompt_data.iloc[0]['input']
ip_keys=preprompt_data.iloc[0]['input_keywords']
ip_summ=preprompt_data.iloc[0]['input_summary']
style_results = preprompt_data.iloc[0]['style_results']
summary_results = preprompt_data.iloc[0]['summary_results']

prompt_to_llm = generate_title_prompt(input_article, style_results, summary_results,ip_keys,ip_summ)
pprint(prompt_to_llm)

In [ ]:
prompt_list=[]
for row in tqdm(range(preprompt_data.shape[0])):
  input_article = preprompt_data.iloc[row]['input']
  ip_keys=preprompt_data.iloc[row]['input_keywords']
  ip_summ=preprompt_data.iloc[row]['input_summary']
  style_results = preprompt_data.iloc[row]['style_results']
  summary_results = preprompt_data.iloc[row]['summary_results']
  prompt_list.append(generate_title_prompt(input_article, style_results, summary_results,ip_keys,ip_summ))


100%|██████████| 100/100 [00:00<00:00, 3433.54it/s]


In [ ]:
preprompt_data['prompts']=prompt_list
preprompt_data.to_pickle('contriever/preprompt_100.pkl')

# RUN PROMPTS THROUGH PALM

In [ ]:
preprompt_data=pd.read_pickle('contriever/preprompt_100.pkl')

In [ ]:
preprompt_data.head(1)

,id,input,profile,output,input_keywords,input_summary,contriever,keyword_results,summary_results,style_results,prompts
1909,301909,Generate a headline for the following article:...,"[{'text': 'Bryant's widow testified that she ""...",33 Phrases To Stop Your Toddler's Tantrums,"tantrums, toddler, parenting, cooperation, smile",Here are some parenting phrases that can help ...,[{'text': 'Parents: We are not there to make l...,[{'text': 'Parents: We are not there to make l...,[{'text': 'Parents: We are not there to make l...,{'statistical_analysis': 'Mean Word Length: 4....,Your task is to analyze the provided data to u...


In [ ]:
import pickle
import time
response_list=[]
for row in tqdm(range(preprompt_data.shape[0])):
  cur_prompt=preprompt_data.iloc[row]['prompts']
  response = palm.generate_text(prompt=cur_prompt)
  response_list.append(response.result)

  if(row%10==0):
    # Save to a pickle file
    with open("prompting_checkpoints/cont_prompt_out"+str(row)+".pkl", "wb") as file:
      pickle.dump(response_list, file)
      time.sleep(30)


preprompt_data['model_output']=response_list
preprompt_data.to_pickle('contriever/postprompt_100.pkl')

100%|██████████| 100/100 [06:53<00:00,  4.14s/it]


In [ ]:
post_prompt_data=pd.read_pickle('contriever/postprompt_100.pkl')

## GETTING UN-PERSONALIZED RESULTS AS WELL

In [ ]:
post_prompt_data.iloc[0]

id                                                            301909
input              Generate a headline for the following article:...
profile            [{'text': 'Bryant's widow testified that she "...
output                    33 Phrases To Stop Your Toddler's Tantrums
bm25_results       [{'text': 'Here are five important tips to hel...
keyword_results    [{'text': 'Here are five important tips to hel...
summary_results    [{'text': 'Here are five important tips to hel...
style_results      {'statistical_analysis': 'Mean Word Length: 4....
input_keywords      tantrums, toddler, parenting, cooperation, smile
input_summary      Here are some parenting phrases that can help ...
prompts            Your task is to analyze the provided data to u...
model_output       5 Parenting Phrases That Will Stop Tantrums In...
Name: 1909, dtype: object

In [ ]:
import pickle
import time
response_list=[]
for row in tqdm(range(post_prompt_data.shape[0])):
  response = palm.generate_text(prompt=post_prompt_data.iloc[row]['input'])
  response_list.append(response.result)

  if(row%10==0):
    time.sleep(20)

post_prompt_data['no_per_model_output']=response_list

100%|██████████| 100/100 [04:04<00:00,  2.44s/it]


In [ ]:
post_prompt_data['no_per_model_output']=response_list

In [ ]:
post_prompt_data.head()

,id,input,profile,output,bm25_results,keyword_results,summary_results,style_results,input_keywords,input_summary,prompts,model_output,no_per_model_output
1909,301909,Generate a headline for the following article:...,"[{'text': 'Bryant's widow testified that she ""...",33 Phrases To Stop Your Toddler's Tantrums,[{'text': 'Here are five important tips to hel...,[{'text': 'Here are five important tips to hel...,[{'text': 'Here are five important tips to hel...,{'statistical_analysis': 'Mean Word Length: 4....,"tantrums, toddler, parenting, cooperation, smile",Here are some parenting phrases that can help ...,Your task is to analyze the provided data to u...,5 Parenting Phrases That Will Stop Tantrums In...,5 Toddler-Taming Phrases That Actually Work
7215,307215,Generate a headline for the following article:...,[{'text': '5. Coming with your significant oth...,Beyonce: Met Gala 2013 Honorary Chair!,[{'text': 'Check out the two covers below. Do ...,[{'text': 'Check out the two covers below. Do ...,[{'text': 'Check out the two covers below. Do ...,{'statistical_analysis': 'Mean Word Length: 5....,"beyonce, gala, chair, costume, institute",Do you think Beyonce is a good choice to co-ch...,Your task is to analyze the provided data to u...,"Beyonce Co-Chairs Met Gala With Anna Wintour, ...",**Beyoncé Co-Chairs Met Gala: Should She Be th...
693,30693,Generate a headline for the following article:...,"[{'text': '""Drivin down the wrong side of the ...",FEARLESS MEMOIR: 'World's Fair' (Chapter 18),[{'text': 'I let my pants drop to the floor an...,[{'text': 'I let my pants drop to the floor an...,[{'text': 'I let my pants drop to the floor an...,{'statistical_analysis': 'Mean Word Length: 4....,"feet, claws, fear, terror, budge",My feet refused to move; they were glued to th...,Your task is to analyze the provided data to u...,None,None
2837,302837,Generate a headline for the following article:...,[{'text': 'He was subdued by passengers and cr...,Buzz Aldrin Looked As Baffled As We Felt Durin...,[{'text': 'It could also be cheeky passcode tr...,[{'text': 'It could also be cheeky passcode tr...,[{'text': 'It could also be cheeky passcode tr...,{'statistical_analysis': 'Mean Word Length: 4....,"infinity, right, don, know, really","""This is infinity here. It could be infinity. ...",Your task is to analyze the provided data to u...,Could This Be The Beginning Of Infinity?,**Could this be infinity?**
10676,3010676,Generate a headline for the following article:...,[{'text': 'Certain points during your day can ...,Starbucks To Stop Throwing Out Perfectly Good ...,[{'text': 'Finding a treatment for sociopaths ...,[{'text': 'Finding a treatment for sociopaths ...,[{'text': 'Finding a treatment for sociopaths ...,{'statistical_analysis': 'Mean Word Length: 4....,"donations, coffee, employees, food, asking",The coffee chain says its employees called for...,Your task is to analyze the provided data to u...,Starbucks Employees Want More Food Donations -...,Starbucks Workers Want More Food Donations: 'T...


In [ ]:
post_prompt_data.to_pickle('postprompt_100.pkl')

# Evaluation

In [ ]:
#*****check on 'contriever/postprompt_100.pkl'

In [ ]:
!pip install rouge

In [ ]:
from transformers import BertTokenizer, BertModel
import torch
import numpy as np
from rouge import Rouge

In [ ]:
def getRougeScores(groundTruth, generatedPrompt):
  rouge = Rouge()
  scores = rouge.get_scores(text1, text2)
  r1, r2, rl = scores[0]['rouge-1']['r'] , scores[0]['rouge-2']['r'] , scores[0]['rouge-l']['r']

  return r1,r2,rl

In [ ]:
def getBertScore(groundTruth, generatedPrompt):
  tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")
  model = BertModel.from_pretrained("bert-base-uncased")

  # Step 1: Prepare the texts for BERT
  inputs1 = tokenizer(groundTruth, return_tensors="pt", padding=True, truncation=True)
  inputs2 = tokenizer(generatedPrompt, return_tensors="pt", padding=True, truncation=True)

  # Step 2: Feed the texts to the BERT model
  outputs1 = model(**inputs1)
  outputs2 = model(**inputs2)

  # Step 3: Obtain the representation vectors
  embeddings1 = outputs1.last_hidden_state.mean(dim=1).detach().numpy()
  embeddings2 = outputs2.last_hidden_state.mean(dim=1).detach().numpy()

  # Step 4: Calculate cosine similarity
  similarity = np.dot(embeddings1, embeddings2.T) / (np.linalg.norm(embeddings1) * np.linalg.norm(embeddings2))

  return similarity[0][0]

#Evaluating News HeadLine Generation

In [ ]:
import pandas as pd
import pickle as pk

from rouge import Rouge
from pandas import DataFrame

In [ ]:
retriever_results = pd.read_pickle(r'contriever/postprompt_100.pkl')

retriever_results.head()

,id,input,profile,output,input_keywords,input_summary,contriever,keyword_results,summary_results,style_results,prompts,model_output
1909,301909,Generate a headline for the following article:...,"[{'text': 'Bryant's widow testified that she ""...",33 Phrases To Stop Your Toddler's Tantrums,"tantrums, toddler, parenting, cooperation, smile",Here are some parenting phrases that can help ...,[{'text': 'Parents: We are not there to make l...,[{'text': 'Parents: We are not there to make l...,[{'text': 'Parents: We are not there to make l...,{'statistical_analysis': 'Mean Word Length: 4....,Your task is to analyze the provided data to u...,5 Parenting Phrases That Will Make Your Toddle...
7215,307215,Generate a headline for the following article:...,[{'text': '5. Coming with your significant oth...,Beyonce: Met Gala 2013 Honorary Chair!,"beyonce, gala, chair, costume, institute",Do you think Beyonce is a good choice to co-ch...,[{'text': 'Editor's Note: Previous references ...,[{'text': 'Editor's Note: Previous references ...,[{'text': 'Editor's Note: Previous references ...,{'statistical_analysis': 'Mean Word Length: 5....,Your task is to analyze the provided data to u...,Beyonce To Co-Chair 2015 Costume Institute Gal...
693,30693,Generate a headline for the following article:...,"[{'text': '""Drivin down the wrong side of the ...",FEARLESS MEMOIR: 'World's Fair' (Chapter 18),"feet, claws, fear, terror, budge",My feet refused to move; they were glued to th...,[{'text': 'All I could think about was Daddy. ...,[{'text': 'All I could think about was Daddy. ...,[{'text': 'All I could think about was Daddy. ...,{'statistical_analysis': 'Mean Word Length: 5....,Your task is to analyze the provided data to u...,None
2837,302837,Generate a headline for the following article:...,[{'text': 'He was subdued by passengers and cr...,Buzz Aldrin Looked As Baffled As We Felt Durin...,"infinity, right, don, know, really","""This is infinity here. It could be infinity. ...","[{'text': '""The article is an absolutely disgu...","[{'text': '""The article is an absolutely disgu...","[{'text': '""The article is an absolutely disgu...",{'statistical_analysis': 'Mean Word Length: 5....,Your task is to analyze the provided data to u...,None
10676,3010676,Generate a headline for the following article:...,[{'text': 'Certain points during your day can ...,Starbucks To Stop Throwing Out Perfectly Good ...,"donations, coffee, employees, food, asking",The coffee chain says its employees called for...,[{'text': 'This is a brutal reminder that glob...,[{'text': 'This is a brutal reminder that glob...,[{'text': 'This is a brutal reminder that glob...,{'statistical_analysis': 'Mean Word Length: 5....,Your task is to analyze the provided data to u...,"Starbucks Employees Want More Food Donations, ..."


In [ ]:
bertScore=[]
for index, row in retriever_results.iterrows() :
  output = row['output']
  modelOp = row['model_output']
  #modelOp = row['no_per_model_output']

  if output != "" and output != None and modelOp != "" and modelOp != None :
    bertScore.append(getBertScore(output, modelOp))
  else :
    bertScore.append(0)


#bertScore

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

In [ ]:
r1lst = []
r2lst = []
rllst = []

rouge = Rouge()

for index, row in retriever_results.iterrows() :
  output = row['output']
  modelOp = row['model_output']
  #modelOp = row['no_per_model_output']

  if output != "" and output != None and modelOp != "" and modelOp != None :
    scores = rouge.get_scores(output, modelOp)
    r1, r2, rl = scores[0]['rouge-1']['r'] , scores[0]['rouge-2']['r'] , scores[0]['rouge-l']['r']
  else:
    r1, r2, rl = 0,0,0

  r1lst.append(r1)
  r2lst.append(r2)
  rllst.append(r1)


In [ ]:
r1lst

In [ ]:
retriever_results['BertScore'] = bertScore
retriever_results['r1-rouge'] = r1lst
retriever_results['r2-rouge'] = r2lst
retriever_results['rl-rouge'] = rllst

In [ ]:
retriever_results['BertScore'] = bertScore

In [ ]:
retriever_results.head()

,id,input,profile,output,input_keywords,input_summary,contriever,keyword_results,summary_results,style_results,prompts,model_output,BertScore,r1-rouge,r2-rouge,rl-rouge
1909,301909,Generate a headline for the following article:...,"[{'text': 'Bryant's widow testified that she ""...",33 Phrases To Stop Your Toddler's Tantrums,"tantrums, toddler, parenting, cooperation, smile",Here are some parenting phrases that can help ...,[{'text': 'Parents: We are not there to make l...,[{'text': 'Parents: We are not there to make l...,[{'text': 'Parents: We are not there to make l...,{'statistical_analysis': 'Mean Word Length: 4....,Your task is to analyze the provided data to u...,5 Parenting Phrases That Will Make Your Toddle...,0.763001,0.222222,0.0,0.222222
7215,307215,Generate a headline for the following article:...,[{'text': '5. Coming with your significant oth...,Beyonce: Met Gala 2013 Honorary Chair!,"beyonce, gala, chair, costume, institute",Do you think Beyonce is a good choice to co-ch...,[{'text': 'Editor's Note: Previous references ...,[{'text': 'Editor's Note: Previous references ...,[{'text': 'Editor's Note: Previous references ...,{'statistical_analysis': 'Mean Word Length: 5....,Your task is to analyze the provided data to u...,Beyonce To Co-Chair 2015 Costume Institute Gal...,0.769582,0.090909,0.0,0.090909
693,30693,Generate a headline for the following article:...,"[{'text': '""Drivin down the wrong side of the ...",FEARLESS MEMOIR: 'World's Fair' (Chapter 18),"feet, claws, fear, terror, budge",My feet refused to move; they were glued to th...,[{'text': 'All I could think about was Daddy. ...,[{'text': 'All I could think about was Daddy. ...,[{'text': 'All I could think about was Daddy. ...,{'statistical_analysis': 'Mean Word Length: 5....,Your task is to analyze the provided data to u...,None,0.000000,0.000000,0.0,0.000000
2837,302837,Generate a headline for the following article:...,[{'text': 'He was subdued by passengers and cr...,Buzz Aldrin Looked As Baffled As We Felt Durin...,"infinity, right, don, know, really","""This is infinity here. It could be infinity. ...","[{'text': '""The article is an absolutely disgu...","[{'text': '""The article is an absolutely disgu...","[{'text': '""The article is an absolutely disgu...",{'statistical_analysis': 'Mean Word Length: 5....,Your task is to analyze the provided data to u...,None,0.000000,0.000000,0.0,0.000000
10676,3010676,Generate a headline for the following article:...,[{'text': 'Certain points during your day can ...,Starbucks To Stop Throwing Out Perfectly Good ...,"donations, coffee, employees, food, asking",The coffee chain says its employees called for...,[{'text': 'This is a brutal reminder that glob...,[{'text': 'This is a brutal reminder that glob...,[{'text': 'This is a brutal reminder that glob...,{'statistical_analysis': 'Mean Word Length: 5....,Your task is to analyze the provided data to u...,"Starbucks Employees Want More Food Donations, ...",0.629246,0.200000,0.0,0.200000


In [ ]:
retriever_results.to_pickle("contriever/metric_output.pkl")

In [ ]:
def average(list):

  return sum(bertScore)/len(bertScore)

0.6073589402437211

In [ ]:
print("Mean r1-rouge {}".format(retriever_results['r1-rouge'].mean()))
print("Mean r2-rouge {}".format(retriever_results['r2-rouge'].mean()))
print("Mean rl-rouge {}".format(retriever_results['rl-rouge'].mean()))
print("Mean bert {}".format(retriever_results['BertScore'].mean()))


Mean r1-rouge 0.1362561849914791
Mean r2-rouge 0.03139023476523477
Mean rl-rouge 0.1362561849914791
Mean bert 0.5802332758903503


In [ ]:
bertSum = 0
r1Sum = 0
r2Sum = 0
NoneCount = 0
r = 0

for index, row in retriever_results.iterrows() :
  if row['model_output'] == None:
    NoneCount += 1
  else:
    bertSum += row['BertScore']
    r1Sum += row['r1-rouge']
    r2Sum += row['r2-rouge']
    r += row['rl-rouge']

noneNull = retriever_results.shape[0] - NoneCount

print("Mean r1", r1Sum/noneNull)
print("Mean r2", r2Sum/noneNull)
print("Mean rl", r/noneNull)
print("Mean bert", bertSum/noneNull)
print("None Count: ", NoneCount)

Mean r1 0.16616607925790133
Mean r2 0.03828077410394483
Mean rl 0.16616607925790133
Mean bert 0.7076015559638419
None Count:  18


In [ ]:
def printScores(df, groundTruthColumn, outputColumn):
  bertScore=[]
  r1lst = []
  r2lst = []
  rllst = []

  bertSum = 0
  r1Sum = 0
  r2Sum = 0
  NoneCount = 0
  r = 0

  rouge = Rouge()


  for index, row in df.iterrows() :
    output = row[groundTruthColumn]
    modelOp = row[outputColumn]

    if output != "" and output != None and modelOp != "" and modelOp != None :
      bs = getBertScore(output, modelOp)
      bertScore.append(bs)
      scores = rouge.get_scores(output, modelOp)
      r1, r2, rl = scores[0]['rouge-1']['r'] , scores[0]['rouge-2']['r'] , scores[0]['rouge-l']['r']

      bertSum += bs
      r1Sum += r1
      r2Sum += r2
      r += rl

    else :
      bertScore.append(0)
      r1, r2, rl = 0,0,0

      NoneCount += 1
  r1lst.append(r1)
  r2lst.append(r2)
  rllst.append(rl)

  metrics = pd.DataFrame(data=[r1lst, r2lst, rllst,bertScore], columns=('r1-rouge', 'R2', 'RL', 'BertScore'))
  print("Mean r1-rouge {}".format(metrics['r1-rouge'].mean()))
  print("Mean r2-rouge {}".format(metrics['r2-rouge'].mean()))
  print("Mean rl-rouge {}".format(metrics['rl-rouge'].mean()))
  # print("Mean bert {}".format(metrics['BertScore'].mean()))


  noneNull = retriever_results.shape[0] - NoneCount

  print("Mean r1", r1Sum/noneNull)
  print("Mean r2", r2Sum/noneNull)
  print("Mean rl", r/noneNull)
  # print("Mean bert", bertSum/noneNull)
  print("None Count: ", NoneCount)

In [ ]:
retriever_results = pd.read_pickle(r'postprompt_100.pkl')

# retriever_results

printScores(retriever_results, 'output', 'no_per_model_output')